In [22]:
using BenchmarkTools
using LinearAlgebra

## Vecchia funzione

In [23]:
function r(args...)
    args = collect(args)
    n = length(args)
    if n == 1 # rotation in 2D
        angle = args[1]; COS = cos(angle); SIN = sin(angle)
        mat = Matrix{Float64}(LinearAlgebra.I, 3, 3)
        mat[1,1] = COS;    mat[1,2] = -SIN;
        mat[2,1] = SIN;    mat[2,2] = COS;
    end

     if n == 3 # rotation in 3D
        mat = Matrix{Float64}(LinearAlgebra.I, 4, 4)
        angle = norm(args);
        if norm(args) != 0.0
			axis = args #normalize(args)
			COS = cos(angle); SIN= sin(angle)
			if axis[2]==axis[3]==0.0    # rotation about x
				mat[2,2] = COS;    mat[2,3] = -SIN;
				mat[3,2] = SIN;    mat[3,3] = COS;
			elseif axis[1]==axis[3]==0.0   # rotation about y
				mat[1,1] = COS;    mat[1,3] = SIN;
				mat[3,1] = -SIN;    mat[3,3] = COS;
			elseif axis[1]==axis[2]==0.0    # rotation about z
				mat[1,1] = COS;    mat[1,2] = -SIN;
				mat[2,1] = SIN;    mat[2,2] = COS;
			else
				I = Matrix{Float64}(LinearAlgebra.I, 3, 3); u = axis
				Ux=[0 -u[3] u[2] ; u[3] 0 -u[1] ;  -u[2] u[1] 1]
				UU =[u[1]*u[1]    u[1]*u[2]   u[1]*u[3];
					 u[2]*u[1]    u[2]*u[2]   u[2]*u[3];
					 u[3]*u[1]    u[3]*u[2]   u[3]*u[3]]
				mat[1:3,1:3]=COS*I+SIN*Ux+(1.0-COS)*UU
			end
		end
	end
	return mat
end

r (generic function with 1 method)

In [24]:
@btime r(0)

  130.233 ns (2 allocations: 256 bytes)


3×3 Array{Float64,2}:
 1.0  -0.0  0.0
 0.0   1.0  0.0
 0.0   0.0  1.0

In [25]:
@btime r(1,1,1)

  834.197 ns (9 allocations: 1.41 KiB)


4×4 Array{Float64,2}:
 1.0      0.17353  2.14758  0.0
 2.14758  1.0      0.17353  0.0
 0.17353  2.14758  1.98703  0.0
 0.0      0.0      0.0      1.0

## Nuove funzioni

In [47]:
function r2D(args)
	angle = args[1]; COS = cos(angle); SIN = sin(angle)
	mat = Matrix{Float64}(LinearAlgebra.I, 3, 3)
	mat[1,1] = COS;    mat[1,2] = -SIN;
	mat[2,1] = SIN;    mat[2,2] = COS;
	return mat
end

r2D (generic function with 1 method)

In [48]:
function rX(mat,COS,SIN)
	mat[2,2] = COS;    mat[2,3] = -SIN;
	mat[3,2] = SIN;    mat[3,3] = COS;
	return mat
end

rX (generic function with 2 methods)

In [49]:
function rY(mat,COS,SIN)
	mat[1,1] = COS;    mat[1,3] = SIN;
	mat[3,1] = -SIN;    mat[3,3] = COS;
	return mat
end

rY (generic function with 2 methods)

In [50]:
function rZ(mat,COS,SIN)
	mat[1,1] = COS;    mat[1,2] = -SIN;
	mat[2,1] = SIN;    mat[2,2] = COS;
	return mat
end

rZ (generic function with 2 methods)

In [51]:
function rAxis(mat,axis,COS,SIN)
	I = Matrix{Float64}(LinearAlgebra.I, 3, 3); u = axis
	Ux=[0 -u[3] u[2] ; u[3] 0 -u[1] ;  -u[2] u[1] 1]
	UU =[u[1]*u[1]    u[1]*u[2]   u[1]*u[3];
		 u[2]*u[1]    u[2]*u[2]   u[2]*u[3];
		 u[3]*u[1]    u[3]*u[2]   u[3]*u[3]]
	mat[1:3,1:3]=COS*I+SIN*Ux+(1.0-COS)*UU
	return mat
end

rAxis (generic function with 2 methods)

In [52]:
function r3D(args)
	mat = Matrix{Float64}(LinearAlgebra.I, 4, 4)
	angle = norm(args);
	if angle != 0.0
		 axis = args #normalize(args)
		 COS = cos(angle); SIN= sin(angle)
		 if axis[2]==axis[3]==0.0    # rotation about x
			 mat = rX(mat,COS,SIN)
		 elseif axis[1]==axis[3]==0.0   # rotation about y
			 mat = rY(mat,COS,SIN)
		 elseif axis[1]==axis[2]==0.0    # rotation about z
			 mat = rZ(mat,COS,SIN)
		 else
			 mat = rAxis(mat,axis,COS,SIN)
		 end
	 end
	 return mat
end

r3D (generic function with 1 method)

In [53]:
function r(args...)
   n = length(args)

   if n == 1 # rotation in 2D
	   mat = r2D(args)
   end

    if n == 3 # rotation in 3D
       mat = r3D(args)
	end
	return mat
end

r (generic function with 1 method)

In [54]:
@btime r(0)

  87.344 ns (1 allocation: 160 bytes)


3×3 Array{Float64,2}:
 1.0  -0.0  0.0
 0.0   1.0  0.0
 0.0   0.0  1.0

In [55]:
@btime r(1,1,1)

  751.969 ns (8 allocations: 1.30 KiB)


4×4 Array{Float64,2}:
 1.0      0.17353  2.14758  0.0
 2.14758  1.0      0.17353  0.0
 0.17353  2.14758  1.98703  0.0
 0.0      0.0      0.0      1.0

In [56]:
@benchmark r(1,1,1)

BenchmarkTools.Trial: 
  memory estimate:  1.30 KiB
  allocs estimate:  8
  --------------
  minimum time:     749.147 ns (0.00% GC)
  median time:      768.974 ns (0.00% GC)
  mean time:        899.143 ns (2.33% GC)
  maximum time:     9.069 μs (81.41% GC)
  --------------
  samples:          10000
  evals/sample:     116

In [57]:
@code_warntype r(1,1,1)

Variables
  #self#::Core.Compiler.Const(r, false)
  args::Tuple{Int64,Int64,Int64}
  n::Int64
  mat::Array{Float64,2}

Body::Array{Float64,2}
1 ─      Core.NewvarNode(:(mat))
│        (n = Main.length(args))
│   %3 = (n::Core.Compiler.Const(3, false) == 1)::Core.Compiler.Const(false, false)
└──      goto #3 if not %3
2 ─      Core.Compiler.Const(:(mat = Main.r2D(args)), false)
3 ┄ %6 = (n::Core.Compiler.Const(3, false) == 3)::Core.Compiler.Const(true, false)
└──      goto #5 if not %6
4 ─      (mat = Main.r3D(args))
5 ┄      return mat


## Modifiche